In [67]:
import sqlite3
DATABASE_PATH = "data.db"
con = sqlite3.connect(DATABASE_PATH)
con.row_factory = sqlite3.Row
cur = con.cursor()

In [2]:
try:
    cur.execute("ALTER TABLE training_runs ADD COLUMN batch_size INTEGER")
except sqlite3.OperationalError as e:
    if "duplicate column name" in str(e).lower():
        print("Column already exists.")
    else:
        raise

con.commit()
con.close()


In [46]:
dataset_value = "820990fd-10-06-2025-1227-47-train"

In [60]:
cur.execute("""
                SELECT datasetSize FROM datasets
                WHERE value = ?
            """, (dataset_value,))

In [61]:
row = cur.fetchall()
data = [dict(row) for row in rows]

In [62]:
print(data)

[{'datasetSize': 120}]


backed up profiles old:

In [6]:
cur.execute("ALTER TABLE profiles RENAME TO profiles_old")


In [9]:
cur.execute("CREATE TABLE profiles("
            "value TEXT PRIMARY KEY,"
            "datasetName TEXT,"
            "datasetSize INTEGER,"
            "description TEXT,"
            "imageHeight INTEGER,"
            "imageWidth INTEGER,"
            "randomOrientation TEXT,"
            "CVPercentage INTEGER,"
            "TestSetPercentage INTEGER)")

In [10]:
cur.execute("INSERT INTO profiles (value, datasetName, datasetSize, description, imageHeight, imageWidth, randomOrientation, CVPercentage, TestSetPercentage)"
"SELECT value, datasetName, datasetSize, description, imageHeight, imageWidth, randomOrientation, CVPercentage, TestSetPercentage"
"FROM profiles_old")

OperationalError: no such column: value

In [6]:
con.commit()
con.close()

In [17]:
cur.execute("PRAGMA table_info(profiles_old)")

In [18]:
print(con.cursor().fetchall())

[]


In [20]:
cur.execute("""
        INSERT INTO profiles (
            value, datasetName, datasetSize, description,
            imageHeight, imageWidth, randomOrientation, CVPercentage
        )
        SELECT 
            value, datasetName, datasetSize, description,
            imageHeight, imageWidth, randomOrientation, CVPercentage
        FROM profiles_old;
    """)

In [21]:
con.commit()

In [22]:
cur.execute("DROP TABLE profiles_old;")

In [23]:
con.commit()

In [24]:
cur.execute("""
        ALTER TABLE datasets ADD COLUMN split TEXT;

""")

In [26]:
con.commit()

In [49]:
cur.execute('DELETE FROM datasets WHERE value = ?', ("4e9ecbf2-15-02-2025-1407-49",))
con.commit()

In [42]:
cur.execute("INSERT INTO datasets ("
                "value, " 
                "datasetName, "
                "datasetSize, "
                "description, "
                "imageHeight, "
                "imageWidth, "
                "randomOrientation )"
                "VALUES (?, ?, ?, ?, ?, ?, ?) ON CONFLICT(value) DO UPDATE SET "
                "datasetName = excluded.datasetName, "
                "datasetSize = excluded.datasetSize, "
                "description = excluded.description, "
                "imageHeight = excluded.imageHeight, "
                "imageWidth = excluded.imageWidth, "
                "randomOrientation = excluded.randomOrientation",
                ("820990fd-21-05-2025-1631-11", 
                "DatasetProf2", 
                1000, 
                "Random orientation. Basic 500x500 dataset of 1000, will initially be used to test checkpoint loading. Test to check generator still works", 
                500, 
                500, 
                1))

In [45]:
con.commit()

## Creating tables for training runs and training data: 

In [19]:
cur.execute("""CREATE TABLE training_runs (
    id TEXT PRIMARY KEY,
    model_id TEXT,
    checkpoint TEXT,
    training_dataset TEXT,
    cv_dataset TEXT,
    test_dataset TEXT,
    optimiser TEXT,
    loss_function TEXT,
    learning_rate REAL,
    epochs INTEGER,
    session_id TEXT
);""")

In [20]:
con.commit()

In [11]:
cur.execute("""CREATE TABLE training_metrics (
    run_id TEXT,
    epoch INTEGER,
    batch INTEGER,
    loss REAL,
    FOREIGN KEY(run_id) REFERENCES training_runs(id)
);""")

In [12]:
con.commit()

In [17]:
cur.execute("""DROP TABLE training_runs""")

In [18]:
con.commit()

Checkpoints table creation:

In [6]:
cur.execute("""CREATE TABLE checkpoints (
    id TEXT PRIMARY KEY,
    model_id TEXT,
    training_run_id TEXT,
    name TEXT,
    notes TEXT,
    final_loss REAL,
    created_at TEXT,
    FOREIGN KEY(model_id) REFERENCES models(value),
    FOREIGN KEY(training_run_id) REFERENCES training_runs(id)
);""")

In [78]:
res = cur.execute("""
            SELECT * FROM training_runs
            ORDER BY id ASC
        """)

In [79]:
row = res.fetchall()

In [80]:
runs_as_dicts = [dict(row) for row in row]

In [81]:
runs_as_dicts = [ dict(row, row['id'])for ro in runs_as_dict]

[{'id': '52f898f6-18-06-2025-1317-23-train',
  'model_id': '52f898f6',
  'checkpoint': '17-06-2025-1211-07.pth',
  'training_dataset': '820990fd-10-06-2025-1227-47-train',
  'cv_dataset': None,
  'test_dataset': None,
  'optimiser': 'ADAM',
  'loss_function': 'MSE',
  'learning_rate': 0.001,
  'epochs': 1,
  'session_id': '18-06-2025-1317-23',
  'batch_size': 20},
 {'id': '52f898f6-18-06-2025-1326-48-train',
  'model_id': '52f898f6',
  'checkpoint': '17-06-2025-1211-07.pth',
  'training_dataset': '820990fd-10-06-2025-1227-47-CV',
  'cv_dataset': None,
  'test_dataset': None,
  'optimiser': 'ADAM',
  'loss_function': 'MSE',
  'learning_rate': 0.001,
  'epochs': 1,
  'session_id': '18-06-2025-1326-48',
  'batch_size': 20},
 {'id': '52f898f6-18-06-2025-1328-40-CV',
  'model_id': '52f898f6',
  'checkpoint': '18-06-2025-1309-52.pth',
  'training_dataset': '820990fd-10-06-2025-1227-47-CV',
  'cv_dataset': None,
  'test_dataset': None,
  'optimiser': None,
  'loss_function': 'MSE',
  'learnin

ValueError: dictionary update sequence element #0 has length 33; 2 is required

In [25]:
con.row_factory = sqlite3.Row

In [26]:
runs_as_dicts = [dict(row) for row in rows]

ValueError: dictionary update sequence element #0 has length 33; 2 is required

In [34]:
import json

In [35]:
try:

        con.row_factory = sqlite3.Row  # enables name-based column access
        cur = con.cursor()
        cur.execute("PRAGMA foreign_keys = ON;")

        cur.execute("""
            SELECT * FROM training_runs
            ORDER BY id ASC
        """)

        rows = cur.fetchall()
        runs_as_dicts = [dict(row) for row in rows]
        print(json.dumps(runs_as_dicts, indent=2))

except sqlite3.Error as e:
    print("Database error:", e)
    print(json.dumps({"error": str(e)}))

finally:
    conn.close()

[
  {
    "id": "52f898f6-18-06-2025-1317-23-train",
    "model_id": "52f898f6",
    "checkpoint": "17-06-2025-1211-07.pth",
    "training_dataset": "820990fd-10-06-2025-1227-47-train",
    "cv_dataset": null,
    "test_dataset": null,
    "optimiser": "ADAM",
    "loss_function": "MSE",
    "learning_rate": 0.001,
    "epochs": 1,
    "session_id": "18-06-2025-1317-23"
  },
  {
    "id": "52f898f6-18-06-2025-1326-48-train",
    "model_id": "52f898f6",
    "checkpoint": "17-06-2025-1211-07.pth",
    "training_dataset": "820990fd-10-06-2025-1227-47-CV",
    "cv_dataset": null,
    "test_dataset": null,
    "optimiser": "ADAM",
    "loss_function": "MSE",
    "learning_rate": 0.001,
    "epochs": 1,
    "session_id": "18-06-2025-1326-48"
  },
  {
    "id": "52f898f6-18-06-2025-1328-40-CV",
    "model_id": "52f898f6",
    "checkpoint": "18-06-2025-1309-52.pth",
    "training_dataset": "820990fd-10-06-2025-1227-47-CV",
    "cv_dataset": null,
    "test_dataset": null,
    "optimiser": null

NameError: name 'conn' is not defined

fetching training run metrics:
    

In [14]:
import json
run_id = "52f898f6-18-06-2025-1317-23-train"
try:
    con.row_factory = sqlite3.Row  # enables name-based column access
    cur = con.cursor()
    cur.execute("PRAGMA foreign_keys = ON;")

    cur.execute("""
        SELECT epoch, batch, loss FROM training_metrics 
        WHERE run_id = ?
        ORDER BY epoch ASC, batch ASC
    """, (run_id,))

    rows = cur.fetchall()
    print(rows)
    
    metrics_as_dicts = [dict(row) for row in rows]
    print(metrics_as_dicts)

    rows_step = [{"step":f"E{row['epoch']}-B{row['batch']}", "loss":row['loss']} for row in metrics_as_dicts]
    print(rows_step)

except sqlite3.Error as e:
    print("Database error:", e)
    print(json.dumps({"error": str(e)})) 



[<sqlite3.Row object at 0x0000027A2B6C69B0>, <sqlite3.Row object at 0x0000027A2B6C73A0>, <sqlite3.Row object at 0x0000027A2B6C63E0>, <sqlite3.Row object at 0x0000027A2B56B8B0>, <sqlite3.Row object at 0x0000027A2A5065C0>, <sqlite3.Row object at 0x0000027A2B57DFF0>]
[{'epoch': 1, 'batch': 20, 'loss': 0.03573734313249588}, {'epoch': 1, 'batch': 40, 'loss': 0.06227292865514755}, {'epoch': 1, 'batch': 60, 'loss': 0.017571741715073586}, {'epoch': 1, 'batch': 80, 'loss': 0.06135644018650055}, {'epoch': 1, 'batch': 100, 'loss': 0.03694869577884674}, {'epoch': 1, 'batch': 120, 'loss': 0.11462163180112839}]
[{'step': 'E1-B20', 'loss': 0.03573734313249588}, {'step': 'E1-B40', 'loss': 0.06227292865514755}, {'step': 'E1-B60', 'loss': 0.017571741715073586}, {'step': 'E1-B80', 'loss': 0.06135644018650055}, {'step': 'E1-B100', 'loss': 0.03694869577884674}, {'step': 'E1-B120', 'loss': 0.11462163180112839}]


## Checkpoint Training Tree:

In [35]:
def training_tree(checkpoint_id):
    training_runs_past = []
    training_runs_future = []
    MAX_DEPTH = 50
    def past_training_runs_query(chkpt_id, depth=0):
        cur = con.cursor()
        if not chkpt_id.strip():
            return
        if depth > MAX_DEPTH:
            print("Max recursion depth hit")
            return
        try:
            print(f"Len == {len(training_runs_past)}")
            cur.execute("""
                SELECT * FROM training_runs
                WHERE INSTR(id, ?) > 0
            """, (chkpt_id,))
                
            rows = cur.fetchall()
            runs_as_dict = [dict(row) for row in rows]
            
            if len(runs_as_dict) == 1:
                
                training_runs_past.append(runs_as_dict[0])
                pprint.pprint(f"Recursing: {chkpt_id}, depth: {depth},  row: ")
                pprint.pprint(runs_as_dict)
                past_training_runs_query(runs_as_dict[0]['checkpoint'], depth+1)
                
        except sqlite3.Error as e:
            print("Database error:", e)
            return json.dumps({"error": str(e)})
    
    past_training_runs_query(checkpoint_id)
    return training_runs_past

In [36]:
ptr = training_tree("26-06-2025-1152-56")

Len == 0
'Recursing: 26-06-2025-1152-56, depth: 0,  row: '
[{'checkpoint': '26-06-2025-1149-20',
  'cv_dataset': None,
  'epochs': 15,
  'id': '52f898f6-26-06-2025-1152-56-train',
  'learning_rate': 0.001,
  'loss_function': 'MSE',
  'model_id': '52f898f6',
  'optimiser': 'ADAM',
  'session_id': '26-06-2025-1152-56',
  'test_dataset': None,
  'training_dataset': '820990fd-10-06-2025-1227-47-train'}]
Len == 1
'Recursing: 26-06-2025-1149-20, depth: 1,  row: '
[{'checkpoint': '26-06-2025-1146-01',
  'cv_dataset': None,
  'epochs': 10,
  'id': '52f898f6-26-06-2025-1149-20-train',
  'learning_rate': 0.001,
  'loss_function': 'MSE',
  'model_id': '52f898f6',
  'optimiser': 'ADAM',
  'session_id': '26-06-2025-1149-20',
  'test_dataset': None,
  'training_dataset': '820990fd-10-06-2025-1227-47-train'}]
Len == 2
'Recursing: 26-06-2025-1146-01, depth: 2,  row: '
[{'checkpoint': '',
  'cv_dataset': None,
  'epochs': 1,
  'id': '52f898f6-26-06-2025-1146-01-train',
  'learning_rate': 0.001,
  'loss

In [37]:
pprint.pprint(ptr)

[{'checkpoint': '26-06-2025-1149-20',
  'cv_dataset': None,
  'epochs': 15,
  'id': '52f898f6-26-06-2025-1152-56-train',
  'learning_rate': 0.001,
  'loss_function': 'MSE',
  'model_id': '52f898f6',
  'optimiser': 'ADAM',
  'session_id': '26-06-2025-1152-56',
  'test_dataset': None,
  'training_dataset': '820990fd-10-06-2025-1227-47-train'},
 {'checkpoint': '26-06-2025-1146-01',
  'cv_dataset': None,
  'epochs': 10,
  'id': '52f898f6-26-06-2025-1149-20-train',
  'learning_rate': 0.001,
  'loss_function': 'MSE',
  'model_id': '52f898f6',
  'optimiser': 'ADAM',
  'session_id': '26-06-2025-1149-20',
  'test_dataset': None,
  'training_dataset': '820990fd-10-06-2025-1227-47-train'},
 {'checkpoint': '',
  'cv_dataset': None,
  'epochs': 1,
  'id': '52f898f6-26-06-2025-1146-01-train',
  'learning_rate': 0.001,
  'loss_function': 'MSE',
  'model_id': '52f898f6',
  'optimiser': 'ADAM',
  'session_id': '26-06-2025-1146-01',
  'test_dataset': None,
  'training_dataset': '820990fd-10-06-2025-1227

In [2]:
import pprint
chkpt_id = "18-06"
try:
#     cur.execute("""
#         SELECT * FROM training_runs
#         WHERE checkpoint = ?
#     """, (chkpt_id,))
    cur.execute("""
        SELECT * FROM training_runs
        WHERE INSTR(id, ?)
    """, (chkpt_id,))
    rows = cur.fetchall()
    runs_as_dict = [dict(row) for row in rows]
    pprint.pprint(runs_as_dict)

except sqlite3.Error as e:
    print("Database error:", e)
    

[{'checkpoint': '17-06-2025-1211-07.pth',
  'cv_dataset': None,
  'epochs': 1,
  'id': '52f898f6-18-06-2025-1317-23-train',
  'learning_rate': 0.001,
  'loss_function': 'MSE',
  'model_id': '52f898f6',
  'optimiser': 'ADAM',
  'session_id': '18-06-2025-1317-23',
  'test_dataset': None,
  'training_dataset': '820990fd-10-06-2025-1227-47-train'},
 {'checkpoint': '17-06-2025-1211-07.pth',
  'cv_dataset': None,
  'epochs': 1,
  'id': '52f898f6-18-06-2025-1326-48-train',
  'learning_rate': 0.001,
  'loss_function': 'MSE',
  'model_id': '52f898f6',
  'optimiser': 'ADAM',
  'session_id': '18-06-2025-1326-48',
  'test_dataset': None,
  'training_dataset': '820990fd-10-06-2025-1227-47-CV'},
 {'checkpoint': '18-06-2025-1309-52.pth',
  'cv_dataset': None,
  'epochs': None,
  'id': '52f898f6-18-06-2025-1328-40-CV',
  'learning_rate': None,
  'loss_function': 'MSE',
  'model_id': '52f898f6',
  'optimiser': None,
  'session_id': '18-06-2025-1328-40',
  'test_dataset': None,
  'training_dataset': '82

In [25]:
import sys
print(sys.version)

3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]


In [41]:
a = "52f898f6-26-06-2025-1152-56-train"
a.findIndex("-")

AttributeError: 'str' object has no attribute 'findIndex'

In [49]:
a[9:27]

'26-06-2025-1152-56'